# Домашнее задание № 9

In [1]:
import accelerate

In [2]:
import os
os.environ["KERAS_BACKEND"] = "torch"

import keras
print(keras.__version__)

3.1.1


In [3]:
# стандартные библиотеки
import os, re
import numpy as np
from time import time
from sklearn.model_selection import train_test_split
import pandas as pd
from collections import Counter
from string import punctuation
import matplotlib.pyplot as plt
%matplotlib inline


import keras, torch
from transformers import AutoModel, AutoModelForMaskedLM
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, AutoModelForTokenClassification

### Задание 1 (5 баллов).
Это задание основано на этой тетрадке - https://github.com/mannefedov/compling_nlp_hse_course/blob/master/notebooks/transfer_learning_hg/Fine_tunining_pretrained_LMs.ipynb

Дообучите 3 предобученных модели на тех же данных, что и в семинаре. Сравните качество и выберите лучшую. 
Модели должны отличаться друг от друга не только параметрами ru/en/multilingual, base/large, cased/uncased. Все три выбранные модели должны работать лучше, чем бейзлайн без дообучения (т.е. если модель не работает, попробуйте другую).


In [4]:
data = pd.read_csv('data/lenta_sample.csv')

In [5]:
data.dropna(subset=['topic', 'text'], inplace=True)
data.topic.value_counts()

topic
Из жизни             55
Наука и техника      54
Бывший СССР          54
Культура             53
Ценности             45
Дом                  45
Бизнес               44
Интернет и СМИ       44
Силовые структуры    40
Спорт                39
Россия               32
Экономика            32
Мир                  27
69-я параллель       13
Легпром              13
Библиотека           10
Крым                  7
Name: count, dtype: int64

In [6]:
def get_device_map() -> str:
    return 'cuda' if torch.cuda.is_available() else 'cpu'

device = get_device_map() 
device

'cpu'

In [7]:
tokenizer = AutoTokenizer.from_pretrained("mrm8488/distilbert-base-multi-cased-finetuned-typo-detection")
model_mrm = AutoModelForTokenClassification.from_pretrained("mrm8488/distilbert-base-multi-cased-finetuned-typo-detection", device_map=device)

In [8]:
X = []

for text in data.title:
    ids = tokenizer.encode(text)
    
    X.append(ids[:512])

X = keras.preprocessing.sequence.pad_sequences(X, padding='post', maxlen=512)

In [9]:
id2label = {i:label for i, label in enumerate(set(data.topic.values))}
label2id = {l:i for i, l in id2label.items()}

y = keras.utils.to_categorical([label2id[label] for label in data.topic.values])

In [10]:
train_index, valid_index = train_test_split(list(range(len(X))), test_size=0.05, stratify=data.topic)

X_train, y_train = X[train_index], y[train_index]
X_valid, y_valid = X[valid_index], y[valid_index]

In [11]:
# если использовать tf как бекенд то заморозить веса можно специальным параметром
# model_bert = AutoModel.from_pretrained('distilbert-base-multilingual-cased', trainable=False)


# в торче веса замораживаются вот так
# если нужно заморозить не все, то нужно просто добавить условия в цикл
for param in model_mrm.parameters():
    param.requires_grad = False

In [12]:
# с huggingface немного сложно, потому что он возвращает тип BaseModelOutput 
# который является tuple как мы посмотрели выше
# и это немного ломает расчет размерностей в керасе потому что он пытается вытащить эту размерность 
# по атрибуту .shape а BaseModelOutput нет такого
# этот атрибут есть у элементов этого tuple поэтому нужно написать вот такую простую обертку
# которая вызовет модель и вернет первый элемент tuple
class model_wrapper(keras.Model):
    def __init__(self, model, **kwargs):
        super().__init__(**kwargs)
        # модель нужно обернуть в специальный класс чтобы она была совместима с другими керас слоями
        self.model = keras.layers.TorchModuleWrapper(model)
        
    def call(self, inputs):
        x = self.model(inputs)
        return x[0]

In [13]:
input_word_ids = keras.layers.Input(shape=(512,), dtype='int32', 
                                     name="input_ids")


output = model_wrapper(model_mrm)(input_word_ids)
# добавим дропаут чтобы не переобучалось
drop = keras.layers.Dropout(0.1)(output[:, 0]) # [0][:, 0] - означает что мы берем первое 
                                                     # состояние у всех текстов в батче
dense = keras.layers.Dense(y.shape[1], activation='softmax')(drop)

model_clf = keras.Model(inputs=[input_word_ids], outputs=dense)

# обычно при дообучении трансформеров нужно ставить очень маленький лосс
# но когда мы обучаем только итоговый классификатор то можно ставить побольше
model_clf.compile(keras.optimizers.Adam(learning_rate=1e-3), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy', 
                           keras.metrics.RecallAtPrecision(0.80, name='rec_prec')])

In [14]:
model_clf.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_ids (InputLayer)          │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ model_wrapper (model_wrapper)   │ (None, 512, 3)         │   134,736,387 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ get_item (GetItem)              │ (None, 3)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 3)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 17)             │            68 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 134,736,455 (513.98 MB)

 Trainable params: 68 (272.00 B)

 Non-trainable params: 134,736,387 (513.98 MB)

In [15]:
if torch.cuda.is_available():
    model_clf.to('cuda')

In [16]:
# если есть ошибка с device meta то попробуйте запустить еще раз
model_clf.fit(torch.LongTensor(X_train), torch.LongTensor(y_train), 
          validation_data=(torch.LongTensor(X_valid), torch.LongTensor(y_valid)),
          batch_size=10,
          epochs=5)

NotImplementedError: Cannot copy out of meta tensor; no data!

In [36]:
def get_device_map() -> str:
    return 'cuda' if torch.cuda.is_available() else 'cpu'

device = get_device_map()  # 'cpu'

base_model = AutoTokenizer.from_pretrained(
        pretrained_model_name_or_path="mrm8488/distilbert-base-multi-cased-finetuned-typo-detection",
        load_in_8bit=False,
        device_map=device)  # HERE
base_model.config.save_pretrained()
base_model.save_pretrained()

AttributeError: 'DistilBertTokenizerFast' object has no attribute 'config'

'cpu'

### Задание 2 (5 баллов).

Это задание основано на этой тетрадке - https://github.com/mannefedov/compling_nlp_hse_course/blob/master/notebooks/gpt/gpt.ipynb

Попробуйте дообучать GPT на каком-то другом тексте (можете попробовать любые стихи или какие-то специфичные вещи вроде анекдотов или репа). 
Попробуйте разные методы и параметры генерации (beam search, температура, top_k и тп). Сохраните в тетрадке несколько хороших сгенерированных текстов. 

Можете использовать другую модель.
